In [38]:
import sys, os, math, numpy as np
import pyproteinsExt.structure.coordinates as PDB
# sys.path.append("DockingPP")
from dockingPP import zParse
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Rotating functions

In [156]:
%autoreload 2
parser = PDB.Parser()
pdbObj = parser.load(file="/Users/jprieto/Docking/data/decoys_bm4_zd3.0.2_6deg_fixed/input_pdbs/1HIA_r_u.pdb.ms_2")

def trans_matrix(psi,theta,phi):
    """Construction de la matrice de transition a partir des angles d'euler"""
    cp=math.cos(psi)
    sp=math.sin(psi)
    ct=math.cos(theta)
    st=math.sin(theta)
    cf=math.cos(phi)
    sf=math.sin(phi)
    d11= cp*cf-sp*ct*sf
    d12=-cp*sf-sp*ct*cf
    d13=sp*st
    
    d21=sp*cf+cp*ct*sf
    d22=-sp*sf+cp*ct*cf
    d23=-cp*st
    
    d31=st*sf
    d32=st*cf
    d33=ct
    tr_matrix = np.matrix([[d11,d12,d13 ],[d21,d22,d23],[d31,d32,d33]])
#     tr_matrix=tr_matrix.T
    return tr_matrix

def coord_matrix(O,P):
    """O,P tuples with 3 values : the x, y and z coordinates of a rotation origin - O
    and of a rotating point or atom - P
    Returns a numpy array with dimensions (1,3) """

    rel_coordinates=np.matrix([[P[i]-O[i]] for i in range(3)])
    return rel_coordinates

def transpose(A,B):
    """A,B two numpy matrixes A a transition matrix, 'matrice de passage'
    B the points to be transferred """
    return A.dot(B)

def recenter(O,P):
    """O,P tuples with 3 values : the x, y and z coordinates of a rotation origin - O
    and of a rotated point or atom - P
    Returns a numpy array with dimensions (1,3) """

    # assert len(O) == len(P) == 3

    rel_coordinates=P+np.matrix([[o] for o in O])
    return rel_coordinates
def translate_rev(O,P):
    """O,P tuples with 3 values : the x, y and z coordinates of a rotation origin - O
    and of a rotated point or atom - P
    Returns a numpy array with dimensions (1,3) """

    # assert len(O) == len(P) == 3

    rel_coordinates=P-np.matrix([[o] for o in O])
    return rel_coordinates



def unmatrix(matrix): 
    t=math.acos(matrix[2,2])
    f=math.asin(matrix[2,0]/math.sin(t))
    p=math.asin(matrix[0,2]/math.sin(t))

    return (p,t,f)
# A revoir l'efficacité de cette fonction
# def is_orthogonal(M):
#     if M.T.all() == M.I.all() :
#         print("This is an orthogonal matrix")
#         return True
#     else :
#         return False


In [157]:
l = [3, 3]
array=np.empty(l)
print(array)
print(array[0,1])

[[ 0.85147086  0.42379542  0.3088605 ]
 [-0.51362971  0.79274681  0.32823317]
 [-0.10574446 -0.43812091  0.89267473]]
0.42379542395370057


In [173]:
DD=zParse("/Users/jprieto/Docking/data/decoys_bm4_zd3.0.2_6deg_fixed/results/1HIA.zd3.0.2.fg.fixed.out")
# print(DD.baryREC)
# print(DD.eulerREC)
DD.setReceptor("/Users/jprieto/Docking/data/decoys_bm4_zd3.0.2_6deg_fixed/input_pdbs/1HIA_r_u.pdb.ms_2")
DD.setLigand("/Users/jprieto/Docking/data/decoys_bm4_zd3.0.2_6deg_fixed/input_pdbs/1HIA_l_u.pdb.ms_2")


# for pose in DD.pList[0:1] : 
#     A=trans_matrix(*DD.eulerREC)
# #     print(A)
#     B=trans_matrix(*pose.euler)
# #     print(B)
#     Mmat=B.dot(A)
#     new_angles=unmatrix(Mmat)
# #     print(Mmat)
# #     print(trans_matrix(*new_angles))
# #     print(DD.eulerREC)
# #     print(pose.euler)
#     pose.euler=new_angles

DD.pList[0].ccmap(dist=5)

    
C=DD.pList[0].resMapList

print(new_angles)
if unmatrix(trans_matrix(*new_angles))==new_angles: 
    print(unmatrix(trans_matrix(*new_angles)))
    print('OK')
else : 
    print(unmatrix(trans_matrix(*new_angles)))

(0.3808886889005771, 1.105860301383092, -0.7705180490419709)
(0.3808886889005771, 1.105860301383092, -0.7705180490419709)
OK


In [177]:
for atom in DD.pdbObjLigand.model[0]:
    initial_coords=(atom.x, atom.y, atom.z)
    a=trans_matrix(*DD.eulerREC)
    b=coord_matrix(DD.baryLIG, initial_coords)
#     print(initial_coords)
#     print(DD.baryLIG)
#     print(b)
    # print(b)
    coords=a.dot(b)
    center_coords=recenter(DD.baryLIG, coords)
    center_coords=translate(DD.baryREC,center_coords)
#     print(coords)
#     print(DD.baryLIG)
#     print(center_coords)
    atom.x=coords[0,0]
    atom.y=coords[1,0]
    atom.z=coords[2,0]
# print(DD.pdbObjLigand)

with open("new_1HIA_lig.pdb",'w') as f: 
    f.write(str(DD.pdbObjLigand))

In [131]:
def where(liste, str_cond) : 
    count=0
    for i in liste : 
        if str_cond in i : 
            count+=1
    return count

print(where(C,'Rec'))
print(where(C,'Lig'))
# print(DD.pList[0]._ccmap)

print(sorted([ i for i in C if 'Rec' in i]))

43
30
['102YRec', '141YRec', '151YRec', '152YRec', '153YRec', '154YRec', '193YRec', '194YRec', '195YRec', '196YRec', '197YRec', '212YRec', '213YRec', '214YRec', '32XRec', '33XRec', '34XRec', '35XRec', '36XRec', '38XRec', '39XRec', '40XRec', '41XRec', '42XRec', '43XRec', '54XRec', '55XRec', '56XRec', '57XRec', '58XRec', '59XRec', '60XRec', '61XRec', '62XRec', '63XRec', '64XRec', '66XRec', '70XRec', '71XRec', '72XRec', '73XRec', '74XRec', '75XRec']


# Test matrices

In [135]:
A=np.matrix([[1,1,1],[2,2,2],[3,3,3]])
X=np.matrix([[1],[2],[3]])
B=np.matrix([[1,2,3],[2,2,3],[4,3,2]])